In [ ]:
import os 
os.environ["OPENAI_API_KEY"] = "Your OpenAI API Key"

In [ ]:
from openai import OpenAI
client = OpenAI()

# ChatGPT Function Define
user_struct = lambda x: {"role": "user", "content": x}
system_struct = lambda x: {"role": "system", "content": x}
assistant_struct = lambda x: {"role": "assistant", "content": x}

servant = "Output the answer without any additional information."

def gpt_answer(messages, model = "gpt-3.5-turbo"):
    response = client.chat.completions.create(
        model = model,
        messages = messages,
        temperature = 0.2,
        timeout = 20
    )
    return response.choices[0].message.content

In [ ]:
import re

def extract_option(response):
    match = re.match(r'^[A-D]', response.strip())
    if match:
        return match.group(0)
    return response

def extract_type(response):
    match = re.match(r'^(Conceptual|Calculation)', response.strip(), re.IGNORECASE)
    if match:
        return match.group(0)
    return response

def extract_numbers_from_string(input_string):
    numbers = re.findall(r'\d+', input_string)

    if numbers:
        return int(numbers[0])
    else:
        return None

In [ ]:
from langchain import hub
from langchain_openai import ChatOpenAI
from langchain.agents import AgentExecutor, create_react_agent
from langchain_experimental.tools import PythonREPLTool

def get_answer_langchain(question_and_choices, model_name = "gpt-3.5-turbo"):
    llm = ChatOpenAI(
        model_name = model_name,
        temperature = 0,
        timeout = 20
    )

    tools = [PythonREPLTool()]
    # PythonREPLTool() may refer to a Python interactive environment (REPL) typically used for interactively executing and testing Python code.
    # This function or object can provide an interactive interface that allows users to input and execute Python code, and immediately see the execution results.

    instructions = """You are an agent designed to answer accounting multiple choice questions.
    You have access to a python REPL, which you can use to execute python code.
    If you get an error, debug your code and try again.
    Only use the output of your code to answer the question.
    You might know the answer without running any code, but you should still run the code to get the answer.
    If it does not seem like you can write code to answer the question, just return "I don't know" as the answer.
    """

    base_prompt = hub.pull("langchain-ai/react-agent-template")
    prompt = base_prompt.partial(instructions=instructions)

    agent = create_react_agent(llm, tools, prompt)
    agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, handle_parsing_errors=True)
    response = agent_executor.invoke({"input": question_and_choices})
    
    try:
        return(response["output"])
    except KeyError:
        return("Error in message chat completions.")

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA

def get_answer_RAG(question_and_choices, vectordb, model_name = "gpt-3.5-turbo"):
    llm = ChatOpenAI(
        model_name = model_name,
        temperature = 0,
        timeout = 20
    )

    retriever = vectordb.as_retriever()
    
    qa = RetrievalQA.from_chain_type(
        llm = llm, 
        chain_type = "stuff", 
        retriever = retriever, 
        verbose = True
    )
    response = qa.invoke({"query": question_and_choices})
    
    try:
        return response["result"]
    except KeyError:
        return "Error in message chat completions."

In [ ]:
def get_answer_RAR(question_and_choices, vectordb, model_name = "gpt-3.5-turbo"):
    llm = ChatOpenAI(
        model_name = model_name,
        temperature = 0,
        timeout = 20
    )

    retriever = vectordb.as_retriever()

    qa = RetrievalQA.from_chain_type(
        llm = llm,
        chain_type = "stuff",
        retriever = retriever,
        verbose = True
    )
    
    rag_response = qa.invoke({"query": question_and_choices})
    
    try:
        additional_info = rag_response["result"]
    except KeyError:
        return "Error in message chat completions (RAG step)."

    tools = [PythonREPLTool()]
    instructions = """You are an agent designed to answer accounting calculation multiple choice questions.
    You have access to a python REPL, which you can use to execute python code.
    If you get an error, debug your code and try again.
    Only use the output of your code to answer the question.
    You might know the answer without running any code, but you should still run the code to get the answer.
    If it does not seem like you can write code to answer the question, just return "I don't know" as the answer.
    """
    
    base_prompt = hub.pull("langchain-ai/react-agent-template")
    prompt = base_prompt.partial(instructions=instructions)

    agent = create_react_agent(llm, tools, prompt)
    agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, handle_parsing_errors=True)
    
    # Combine the additional info with the original question and choices
    print(additional_info)
    combined_input = f"{question_and_choices}\n\nAdditional Information:\n{additional_info}"
    response = agent_executor.invoke({"input": combined_input})
    
    try:
        return response["output"]
    except KeyError:
        return "Error in message chat completions (Python REPL step)."


In [ ]:
######################################################################
### These steps only need to run once for building vector database ###
######################################################################

from langchain.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyMuPDFLoader

# the following are loading textbooks' PDF
loader1 = PyMuPDFLoader("Text book 1")
documents1 = loader1.load()
loader2 = PyMuPDFLoader("Text book 2")
documents2 = loader2.load()
loader3 = PyMuPDFLoader("Text book 3")
documents3 = loader3.load()
loader4 = PyMuPDFLoader("Text book 4")
documents4 = loader4.load()
# ......

documents = documents1 + documents2 + documents3 + documents4 

# split document into small chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 200,   # max size of each chunk
    chunk_overlap = 10   # overlap to ensure context continuity
)
chunks = text_splitter.split_documents(documents)

# Create embeddings
embedding = OpenAIEmbeddings() # use OpenAi's embedding model

# Persist embeddings to disk
persist_directory = 'book_db'
vectordb = Chroma.from_documents(documents = chunks, embedding = embedding, persist_directory = persist_directory)

In [ ]:
############################################################################################
### Once build the vector database, you only need to run these code for next experiments ###
############################################################################################

from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma

persist_directory = 'book_db'
embedding = OpenAIEmbeddings()
vectordb = Chroma(persist_directory = persist_directory, embedding_function = embedding) # load stored vector database 

In [ ]:
import json

file_path = "./CPA_example.json" # data path for questions
with open(file_path, 'r', encoding='utf-8') as file:
    data = json.load(file)

In [ ]:
import time

question_type = {"AUD": [0, 0], "BEC": [0, 0], "FAR": [0, 0], "REG": [0, 0]}
correct_answer = 0

for index, item in enumerate(data["questions"]):
    
    id = item["question_id"]
    question = item['question']
    choices = [item['choice_a'], item['choice_b'], item['choice_c'], item['choice_d']]
    answer = item['answer']
    
    # Task router: determine question type 
    type_prompt = "I have a CPA exam question, and I need to know whether it is a calculation question or a conceptual question. Here is the question:\n"
    type_prompt += f"{question}\n A. {choices[0]}\n B. {choices[1]}\n C. {choices[2]}\n D. {choices[3]}"
    type_prompt += "\nCan you please determine if this is a calculation question or a conceptual question?"
    messages = [
        system_struct(servant),
        user_struct(type_prompt)
    ]
    type_response = gpt_answer(messages).strip()
    Type = extract_type(type_response)
    # print(f"{Type}")
    
    if "Conceptual" in Type or "conceptual" in Type:
        prompt = "This is the conceptual question of CPA exam, please think that:\n"
        prompt += f"{question}\n A. {choices[0]}\n B. {choices[1]}\n C. {choices[2]}\n D. {choices[3]}"
        prompt += "\nWhat is the correct option? Output the answer without any additional information."
        LLM_response = get_answer_RAG(prompt, vectordb, model_name="gpt-4o-mini").strip() # you can change default openAI model for others
    else: # Calculation
        prompt = "Let's think step by step:\n"
        prompt += f"{question}\n A. {choices[0]}\n B. {choices[1]}\n C. {choices[2]}\n D. {choices[3]}"
        prompt += "\nThe correct answer for previous options is A, B, C, or D?\n"
        LLM_response = get_answer_RAR(prompt, vectordb, model_name="gpt-4o-mini").strip() # you can change default openAI model for others
            
    extracted_option = extract_option(LLM_response)
    print(f"{id} answer: {extracted_option}, correct answer: {answer}")
    
    if(extracted_option == answer):
        print("Correct!!")
        correct_answer += 1
        question_type[item["type"]][0] += 1
    else:
        print("Wrong!!")
    
    question_type[item["type"]][1] += 1
    # print(question_type)
            
    time.sleep(2)

In [ ]:
print(question_type["AUD"])
print(f"AUD: {question_type['AUD'][0] / question_type['AUD'][1]}")

print(question_type["BEC"])
print(f"BEC: {question_type['BEC'][0] / question_type['BEC'][1]}")

print(question_type["FAR"])
print(f"FAR: {question_type['FAR'][0] / question_type['FAR'][1]}")

print(question_type["REG"])
print(f"REG: {question_type['REG'][0] / question_type['REG'][1]}")

total_question  = question_type["AUD"][1] + question_type["BEC"][1] + question_type["FAR"][1] + question_type["REG"][1]
print(f"\nTotal accuracy: {correct_answer / total_question}")